<a href="https://colab.research.google.com/github/abdn-cs3033-ai/practicals/blob/main/week03/tutorial2-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS3033: Artificial Intelligence

## Tutorial 02: Search Algorithms

<h4>Prof. Felipe Meneguzzi</h4>

In order to run this tutorial, you need to download the auxiliary files from Github into your notebook, which we do with Jupyter's shell commands (if you downloaded the entire repo, the code below is not necessary).

In [5]:
try:
    import google.colab
    print("We are in Google colab, we need to clone the repo")
    !git clone https://github.com/abdn-cs3033-ai/practicals.git
    %cd practicals/week03
    %pip install -r requirements.txt
except:
    print("Not in colab")

Not in colab


## Navigation domain

The problem we want to address in this domain is navigation in a grid-like environment. The dynamics of this domain is in ```player.py```, and we encode the maps as text files in ```maps/*.txt```.
Below we show how to display an 'easy' map, where *S* denotes the starting tile and the circle denotes the goal tile. 

Maps are [standard text files](maps/easy.txt) where specific characters represent obstacles (`1`), empty spaces (`0`), and the goal (`2`). The first two lines of the text file encode the coordinates of the starting point. We already provide a standardised interface for maps in the Map class within [`common.py`](common.py). In that class, the key method you will use to generate successors to the current position you are search is `successors`.

In [6]:
from IPython.display import HTML, display
import common
from common import Point, Map, read_map, TILE_CLEAR, TILE_CLOSED, TILE_GOAL

def display_map(map: Map):
    map_html='<table style="font-size:300%;border: thick solid;">'
    for i in range(map.height):
        map_html+='<tr>'
        for j in range(map.width):
            map_html+='<td>'
            if map.data[j][i]==TILE_CLOSED:
                map_html+='&#x25FE;'
            elif map.data[j][i]==TILE_GOAL:
                map_html+='&#x25CE;'
            elif i==map.start[0] and j==map.start[1]:
                map_html+='S'
            map_html+='</td>'
        map_html+='</tr>'
    map_html+='</tr></table>'

    display(HTML(map_html))

map = read_map('easy.txt')
display_map(map)

◾,◾,◾,◾,◾,◾,◾,◾,◾,◾
◾,◾,◾,◾,,◾,◾,◾,◾,◾
◾,◾,◾,◾,,◾,◾,◾,◾,◾
◾,◾,◾,◾,,◾,◾,◾,◾,◾
◾,◾,,,,,,,S,◾
◾,◾,,◾,,◾,◾,◾,◾,◾
◾,◾,,◾,,◾,◾,◾,◾,◾
◾,◾,◎,◾,◾,◾,◾,◾,◾,◾
◾,◾,◾,◾,◾,◾,◾,◾,◾,◾
◾,◾,◾,◾,◾,◾,◾,◾,◾,◾


# Overview

For this ungraded tutorial, you will implement the $A^{*}$ heuristic search algorithm to solve instances of the navigation problem. Given a board state, find a combination of moves that leads to the final state. As an informed search method, $A^{*}$, relies on a heuristic distance function in order to efficiently prune the state space and speed up the search tree expansion towards the solution. In class, we saw two possible heuristics that can be applied to this problem, but in this assignment, we will focus on the Manhattan Distance Heuristic. This heuristic consists of distance in orthogonal movements between the block and its desired destination. 

**Note:** Do not share the code from your practical to other colleagues (or post it on the internet), as this code will be useful for the programming assignment.

## Implementation

You need to implement only the API below, below. However, you may create individual classes to represent elements in the state space as well as nodes in the search tree. Alternatively, you can implement most of the internals using [Python tuples](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences). We note that the API shown below is deliberately simplified to give students the maximum freedom to develop their own internal APIs and minimize coincidental similarities in implementation. 

Get familiar with Python's [Priority Queue](https://docs.python.org/3/library/queue.html#queue.PriorityQueue), since this is the data structure you need to implement the frontiers. You are welcome to implement your own priority queue for the practical.

In [7]:
from queue import PriorityQueue
from common import Point, Map, read_map, direction, DEFAULT_MAP, MOVE_DOWN, MOVE_LEFT, MOVE_RIGHT, MOVE_UP

class PathFinder_A_Star:
    def __init__(self):
        # TODO initialize your attributes here if needed
        pass
    
    def heuristic(self, p1: Point, p2: Point):
        # TODO heuristic function
        return 0
    
    def solve(self, map: Map):
        # TODO returns a list of movements (may be empty)
        # if plan found, otherwise returns None
        return None

    def get_solvable(self, map: Map):
        # TODO returns True if plan found,
        # otherwise returns False
        return False
    
    def get_max_tree_height(self, map: Map):
        # TODO returns max tree height if plan found,
        # otherwise returns None
        return None

    def get_min_moves(self, map: Map):
        # TODO returns size of minimal plan to reach goal if plan found,
        # otherwise returns None
        return None

## Code-Specific Advice

- In ```common.py```, you can change the current map modifying the ```DEFAULT_MAP``` constant;
<!-- - You can also change the block's movement speed modifying the ```MOVE_SPEED``` constant in the same file; -->
- All classes and methods which you must complete yourself are marked with the ```TODO``` keyword;

## Testing

You can automatically test your solution code with the unit tests below.

In [8]:
import unittest
import sys
from common import *

try:
    from timeout_decorator import timeout
except:
    print ("""Oops! It looks like you don\'t have timeout-decorator installed. Please do so using the following command:
        pip install timeout-decorator""")
    sys.exit(0)

# Map definitions
TRIVIAL = "trivial.txt"
TRIVIAL_SOLUTION = []
TRIVIAL_MAX_TREE_HEIGHT = 0
TRIVIAL_GRADE = 0.25

EASY = "easy.txt"
EASY_SOLUTION = [
    MOVE_UP, MOVE_UP, MOVE_UP, MOVE_UP, MOVE_UP, MOVE_UP,
    MOVE_RIGHT, MOVE_RIGHT, MOVE_RIGHT]
EASY_MAX_TREE_HEIGHT = 9 # ToDo find max tree height correct value
EASY_GRADE = 0.25

MEDIUM = "medium.txt"
MEDIUM_SOLUTION = [
    MOVE_DOWN, MOVE_DOWN, MOVE_DOWN, MOVE_DOWN, MOVE_DOWN, MOVE_DOWN, MOVE_DOWN,
    MOVE_RIGHT, MOVE_RIGHT, MOVE_RIGHT, MOVE_RIGHT, MOVE_RIGHT,
    MOVE_UP, MOVE_UP, MOVE_UP,
    MOVE_RIGHT, MOVE_RIGHT, MOVE_RIGHT,
    MOVE_UP, MOVE_UP,
    MOVE_LEFT, MOVE_LEFT, MOVE_LEFT,
    MOVE_UP, MOVE_UP,
    MOVE_LEFT, MOVE_LEFT, MOVE_LEFT]
MEDIUM_MAX_TREE_HEIGHT = 28 # ToDo find max tree height correct value
MEDIUM_GRADE = 0.25


# ==========================================
# Test A*
# ==========================================

class Test_A_Star(unittest.TestCase):

    # ------------------------------------------
    # Setup
    # ------------------------------------------

    @classmethod
    def setUpClass(cls):
        cls.grade = 0.0
        cls.total = 0.0

    @classmethod
    def tearDownClass(cls):
        print ("  Grade: ", cls.grade, " of ", cls.total)

    # ------------------------------------------
    # Common tests
    # ------------------------------------------

    def solvable(self, map_name, expected):
        map = read_map(map_name)
        pathfinder = PathFinder_A_Star()
        pathfinder.solve(map)
        #self.assertEqual(pathfinder.get_solvable(map), expected)
        var1 = pathfinder.get_solvable(map)
        var2 = expected
        if var1 != var2:
            print ("solvable failed! pathfinder.get_solvable(map) is not equal to expected\n")

    def plan_match(self, map_name, moves):
        plan = PathFinder_A_Star().solve(read_map(map_name))
        #self.assertEqual(plan, moves)
        var1 = plan
        var2 = moves
        if var1 != var2:
            print ("plan_match failed! plan is not equal to moves\n")

    def max_tree_height(self, map_name, height):
        map = read_map(map_name)
        pathfinder = PathFinder_A_Star()
        plan = pathfinder.solve(map)
        #self.assertEqual(pathfinder.get_max_tree_height(map), height)
        var1 = pathfinder.get_max_tree_height(map)
        var2 = height
        if var1 != var2:
            print ("max_tree_height failed! pathfinder.get_max_tree_height(map) is not equal to height\n")

    def min_moves(self, map_name, moves):
        map = read_map(map_name)
        pathfinder = PathFinder_A_Star()
        plan = pathfinder.solve(map)
        if moves != None:
            #self.assertEqual(pathfinder.get_min_moves(map), len(moves))
            var1 = pathfinder.get_min_moves(map)
            var2 = len(moves)
            if var1 != var2:
                print ("min_moves failed! pathfinder.get_min_moves(map) is not equal to len(moves)\n")

        else:
            #self.assertEqual(pathfinder.get_min_moves(map), moves)
            var1 = pathfinder.get_min_moves(map)
            var2 = moves
            if var1 != var2:
                print ("min_moves failed! pathfinder.get_min_moves(map) is not equal to moves\n")


    # =============
    # = Heuristic =
    # =============
    
    def test_heuristic(self):
        pathfinder = PathFinder_A_Star()
        p1 = Point(1, 2)
        p2 = Point(1, 3)
        #self.assertEqual(pathfinder.heuristic(p1, p2),1)
        var1 = pathfinder.heuristic(p1, p2)
        var2 = 1
        if var1 != var2:
            print ("test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 1\n")
        
        p1 = Point(2, 2)
        p2 = Point(1, 3)
        #self.assertEqual(pathfinder.heuristic(p1, p2),2)
        var1 = pathfinder.heuristic(p1, p2)
        var2 = 2
        if var1 != var2:
            print ("test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 2\n")
            
        p1 = Point(20, 2)
        p2 = Point(1, 2)
        #self.assertEqual(pathfinder.heuristic(p1, p2),19)
        var1 = pathfinder.heuristic(p1, p2)
        var2 = 19
        if var1 != var2:
            print ("test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 19\n")

    # ------------------------------------------
    # Trivial
    # ------------------------------------------
    @timeout(15)
    def test_trivial_solvable(self):
        self.__class__.total += TRIVIAL_GRADE
        self.solvable(TRIVIAL, True)
        self.__class__.grade += TRIVIAL_GRADE

    @timeout(15)
    def test_trivial_plan_match(self):
        self.__class__.total += TRIVIAL_GRADE
        self.plan_match(TRIVIAL, TRIVIAL_SOLUTION)
        self.__class__.grade += TRIVIAL_GRADE

    @timeout(15)
    def test_trivial_max_tree_height(self):
        self.__class__.total += TRIVIAL_GRADE
        self.max_tree_height(TRIVIAL, TRIVIAL_MAX_TREE_HEIGHT)
        self.__class__.grade += TRIVIAL_GRADE

    @timeout(15)
    def test_trivial_min_moves(self):
        self.__class__.total += TRIVIAL_GRADE
        self.min_moves(TRIVIAL, TRIVIAL_SOLUTION)
        self.__class__.grade += TRIVIAL_GRADE

    # ------------------------------------------
    # Easy
    # ------------------------------------------
    @timeout(15)
    def test_easy_solvable(self):
        self.__class__.total += EASY_GRADE
        self.solvable(EASY, True)
        self.__class__.grade += EASY_GRADE

    @timeout(15)
    def test_easy_plan_match(self):
        self.__class__.total += EASY_GRADE
        self.plan_match(EASY, EASY_SOLUTION)
        self.__class__.grade += EASY_GRADE

    @timeout(15)
    def test_easy_max_tree_height(self):
        self.__class__.total += EASY_GRADE
        self.max_tree_height(EASY, EASY_MAX_TREE_HEIGHT)
        self.__class__.grade += EASY_GRADE

    @timeout(15)
    def test_easy_min_moves(self):
        self.__class__.total += EASY_GRADE
        self.min_moves(EASY, EASY_SOLUTION)
        self.__class__.grade += EASY_GRADE

    # ------------------------------------------
    # Medium
    # ------------------------------------------
    @timeout(15)
    def test_medium_solvable(self):
        self.__class__.total += MEDIUM_GRADE
        self.solvable(MEDIUM, True)
        self.__class__.grade += MEDIUM_GRADE

    @timeout(15)
    def test_medium_plan_match(self):
        self.__class__.total += MEDIUM_GRADE
        self.plan_match(MEDIUM, MEDIUM_SOLUTION)
        self.__class__.grade += MEDIUM_GRADE

    @timeout(15)
    def test_medium_max_tree_height(self):
        self.__class__.total += MEDIUM_GRADE
        self.max_tree_height(MEDIUM, MEDIUM_MAX_TREE_HEIGHT)
        self.__class__.grade += MEDIUM_GRADE

    @timeout(15)
    def test_medium_min_moves(self):
        self.__class__.total += MEDIUM_GRADE
        self.min_moves(MEDIUM, MEDIUM_SOLUTION)
        self.__class__.grade += MEDIUM_GRADE

        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    #unittest.main()



.............

max_tree_height failed! pathfinder.get_max_tree_height(map) is not equal to height

min_moves failed! pathfinder.get_min_moves(map) is not equal to len(moves)

plan_match failed! plan is not equal to moves

solvable failed! pathfinder.get_solvable(map) is not equal to expected

test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 1

test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 2

test_heuristic failed! pathfinder.heuristic(p1, p2) is not equal to 19

max_tree_height failed! pathfinder.get_max_tree_height(map) is not equal to height

min_moves failed! pathfinder.get_min_moves(map) is not equal to len(moves)

plan_match failed! plan is not equal to moves

solvable failed! pathfinder.get_solvable(map) is not equal to expected

max_tree_height failed! pathfinder.get_max_tree_height(map) is not equal to height

min_moves failed! pathfinder.get_min_moves(map) is not equal to len(moves)

plan_match failed! plan is not equal to moves

solvable failed! path


----------------------------------------------------------------------
Ran 13 tests in 0.009s

OK
